## Data Preparation

This notebook prepares the data for ML modeling by:
1. Loading raw data
2. Replicating EDA feature engineering
3. Selecting features for ML
4. Saving processed dataset


In [12]:
import pandas as pd
import numpy as np
from pathlib import Path

# Set up paths
project_root = Path().resolve().parent
data_dir = project_root / "data"
processed_dir = data_dir / "processed"
processed_dir.mkdir(exist_ok=True)

print(f"Project root: {project_root}")
print(f"Data directory: {data_dir}")
print(f"Processed directory: {processed_dir}")


Project root: /Users/qadeermac/workspace/ML-Toronto-Shelter-Occupancy /ML-Toronto-Shelter-Occupancy
Data directory: /Users/qadeermac/workspace/ML-Toronto-Shelter-Occupancy /ML-Toronto-Shelter-Occupancy/data
Processed directory: /Users/qadeermac/workspace/ML-Toronto-Shelter-Occupancy /ML-Toronto-Shelter-Occupancy/data/processed


In [13]:
# Load raw data
print("Loading raw data...")
df = pd.read_csv(data_dir / "sheltersdata.csv")

# Convert date
df['OCCUPANCY_DATE'] = pd.to_datetime(df['OCCUPANCY_DATE'], format='mixed')

print(f"Loaded {len(df)} rows")
print(f"Date range: {df['OCCUPANCY_DATE'].min()} to {df['OCCUPANCY_DATE'].max()}")
df.head()


Loading raw data...
Loaded 242658 rows
Date range: 2001-01-21 00:00:00 to 2031-12-22 00:00:00


,_id,OCCUPANCY_DATE,ORGANIZATION_ID,ORGANIZATION_NAME,SHELTER_ID,SHELTER_GROUP,LOCATION_ID,LOCATION_NAME,LOCATION_ADDRESS,LOCATION_POSTAL_CODE,...,OCCUPIED_BEDS,UNOCCUPIED_BEDS,UNAVAILABLE_BEDS,CAPACITY_ACTUAL_ROOM,CAPACITY_FUNDING_ROOM,OCCUPIED_ROOMS,UNOCCUPIED_ROOMS,UNAVAILABLE_ROOMS,OCCUPANCY_RATE_BEDS,OCCUPANCY_RATE_ROOMS
0,1,2025-01-01,24,COSTI Immigrant Services,40,COSTI Reception Centre,1320.0,COSTI Hotel Program Dixon,640 Dixon Rd.,M9W 1J1,...,NaN,NaN,NaN,148.0,150.0,148.0,0.0,2.0,NaN,100.0
1,2,2025-01-01,24,COSTI Immigrant Services,40,COSTI Reception Centre,1320.0,COSTI Hotel Program Dixon,640 Dixon Rd.,M9W 1J1,...,NaN,NaN,NaN,57.0,57.0,57.0,0.0,0.0,NaN,100.0
2,3,2025-01-01,24,COSTI Immigrant Services,40,COSTI Reception Centre,1051.0,COSTI Reception Centre,100 Lippincott St,M5S 2P1,...,8.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,100.0,NaN
3,4,2025-01-01,24,COSTI Immigrant Services,40,COSTI Reception Centre,1051.0,COSTI Reception Centre,100 Lippincott St,M5S 2P1,...,8.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,100.0,NaN
4,5,2025-01-01,24,COSTI Immigrant Services,40,COSTI Reception Centre,1114.0,COSTI Uptown Hotel Program,55 Hallcrown Pl,M2J 4R1,...,NaN,NaN,NaN,70.0,83.0,70.0,0.0,13.0,NaN,100.0


In [14]:
# Drop rows missing the regression target
print("Dropping rows with missing OCCUPIED_BEDS...")
df = df.dropna(subset=['OCCUPIED_BEDS'])

# Classification target
df['overcapacity'] = (df['OCCUPANCY_RATE_BEDS'] > 95).astype(int)

print(f"After dropping missing values: {len(df)} rows")
print(f"Overcapacity rate: {df['overcapacity'].mean():.2%}")


Dropping rows with missing OCCUPIED_BEDS...
After dropping missing values: 165802 rows
Overcapacity rate: 81.22%


In [15]:
# Temporal Features
print("Creating temporal features...")
df['day_of_week'] = df['OCCUPANCY_DATE'].dt.dayofweek
df['month'] = df['OCCUPANCY_DATE'].dt.month
df['week_of_year'] = df['OCCUPANCY_DATE'].dt.isocalendar().week


Creating temporal features...


In [16]:
# Lag & Rolling Features
print("Creating lag and rolling features...")
df = df.sort_values(['SHELTER_ID', 'OCCUPANCY_DATE'])
df['lag_1'] = df.groupby('SHELTER_ID')['OCCUPIED_BEDS'].shift(1)
df['lag_7'] = df.groupby('SHELTER_ID')['OCCUPIED_BEDS'].shift(7)
df['roll_mean_7'] = df.groupby('SHELTER_ID')['OCCUPIED_BEDS'].rolling(7).mean().shift(1).reset_index(0, drop=True)


Creating lag and rolling features...


In [17]:
# Encode categorical features as bools
print("Encoding categorical features...")
cat_cols = ['SECTOR', 'PROGRAM_MODEL', 'OVERNIGHT_SERVICE_TYPE', 'PROGRAM_AREA']
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)
print(f"After encoding: {df.shape[1]} columns")


Encoding categorical features...
After encoding: 52 columns


In [18]:
# Cyclical encoding for temporal features
print("Creating cyclical features...")
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['week_of_year_sin'] = np.sin(2 * np.pi * df['week_of_year'] / 52)
df['week_of_year_cos'] = np.cos(2 * np.pi * df['week_of_year'] / 52)


Creating cyclical features...


In [19]:
# Handle missing values for lag features
print("Handling missing values...")
df.fillna(0, inplace=True)
print("Missing values handled")


Handling missing values...
Missing values handled


In [20]:
# Select features for ML
print("Selecting features for ML...")

# Features to drop (metadata and raw temporal features)
drop_cols = [
    '_id', 'OCCUPANCY_DATE',
    'ORGANIZATION_NAME', 'SHELTER_GROUP', 'LOCATION_NAME',
    'LOCATION_ADDRESS', 'LOCATION_POSTAL_CODE', 'LOCATION_CITY', 'LOCATION_PROVINCE',
    'PROGRAM_NAME', 'CAPACITY_TYPE',
    'day_of_week', 'month', 'week_of_year',  # Use cyclical versions instead
    'UNOCCUPIED_BEDS', 'UNAVAILABLE_BEDS',  # These are components of OCCUPANCY_RATE_BEDS
    'CAPACITY_ACTUAL_ROOM', 'CAPACITY_FUNDING_ROOM', 'OCCUPIED_ROOMS',
    'UNOCCUPIED_ROOMS', 'UNAVAILABLE_ROOMS', 'OCCUPANCY_RATE_ROOMS'  # Room-based, not bed-based
]
# Note: Keep OCCUPIED_BEDS for lag feature calculation in API

# Keep all other columns as features (exclude OCCUPIED_BEDS as it's highly correlated with target)
feature_cols = [col for col in df.columns if col not in drop_cols and col not in ['OCCUPANCY_RATE_BEDS', 'overcapacity', 'OCCUPIED_BEDS']]

print(f"Number of features: {len(feature_cols)}")
print(f"Feature columns: {feature_cols[:10]}..." if len(feature_cols) > 10 else f"Feature columns: {feature_cols}")


Selecting features for ML...
Number of features: 33
Feature columns: ['ORGANIZATION_ID', 'SHELTER_ID', 'LOCATION_ID', 'PROGRAM_ID', 'SERVICE_USER_COUNT', 'CAPACITY_ACTUAL_BED', 'CAPACITY_FUNDING_BED', 'lag_1', 'lag_7', 'roll_mean_7']...


In [21]:
# Create feature dataset (include OCCUPIED_BEDS for API lag feature calculation)
features_df = df[['SHELTER_ID', 'OCCUPANCY_DATE', 'OCCUPIED_BEDS'] + feature_cols + ['OCCUPANCY_RATE_BEDS', 'overcapacity']].copy()

print(f"\nDataset shape: {features_df.shape}")
print(f"Columns: {features_df.columns.tolist()[:5]}...")



Dataset shape: (165802, 38)
Columns: ['SHELTER_ID', 'OCCUPANCY_DATE', 'OCCUPIED_BEDS', 'ORGANIZATION_ID', 'SHELTER_ID']...


In [22]:
# Save processed dataset
output_path = processed_dir / "features.csv"
print(f"\nSaving processed data to {output_path}...")
features_df.to_csv(output_path, index=False)

# Save feature list for later use
feature_list_path = processed_dir / "feature_list.txt"
with open(feature_list_path, 'w') as f:
    f.write('\n'.join(feature_cols))

print("✓ Data preparation complete!")
print(f"✓ Saved to {output_path}")
print(f"✓ Feature list saved to {feature_list_path}")



Saving processed data to /Users/qadeermac/workspace/ML-Toronto-Shelter-Occupancy /ML-Toronto-Shelter-Occupancy/data/processed/features.csv...
✓ Data preparation complete!
✓ Saved to /Users/qadeermac/workspace/ML-Toronto-Shelter-Occupancy /ML-Toronto-Shelter-Occupancy/data/processed/features.csv
✓ Feature list saved to /Users/qadeermac/workspace/ML-Toronto-Shelter-Occupancy /ML-Toronto-Shelter-Occupancy/data/processed/feature_list.txt
